Limpieza de base de datos

-----------------

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
#Rutas de las bases de datos desde el Drive

import os

if os.path.exists('/content/drive/My Drive/PI 2.0/multilabel_CSV.csv'):
    file_path = '/content/drive/My Drive/PI 2.0/multilabel_CSV.csv'
elif os.path.exists('/content/drive/Shared with me/PI 2.0/multilabel_CSV.csv'):
    file_path = '/content/drive/Shared with me/PI 2.0/multilabel_CSV.csv'
else:
    raise FileNotFoundError("No se encontró el archivo en ninguna ruta conocida.")

file_path_starpep = '/content/drive/My Drive/PI 2.0/Function_metadata.csv'
file_path_excel = '/content/drive/My Drive/PI 2.0/multilabel_Excel.xlsx'


file_merged_starpep = '/content/drive/My Drive/PI 2.0/merged-starpep.csv'
file_merged_unal = '/content/drive/My Drive/PI 2.0/merged-unal.csv'


In [5]:
#!pip install pandas openpyxl
import pandas as pd

df_unal= pd.read_csv(file_path, delimiter=',')

df_excel_antitumoral= pd.read_excel(file_path_excel, sheet_name='Antitumoral')
df_excel_antitumoral=df_excel_antitumoral.drop(columns=['Secuencias raras, con errores o vacías', 'Unnamed: 3', 'ID'])

df_excel_antivih= pd.read_excel(file_path_excel, sheet_name='Anti-VIH')
df_excel_antivih=df_excel_antivih.drop(columns=['Secuencias raras, con errores o vacías', 'Unnamed: 3', 'ID'])

df_unal = df_unal.drop(columns=['aliphatic_index','aromaticity','boman_index','charge','charge_density','class', 'gravy', 'hydrophobic_ratio', 'instability_index', 'isoelectric_point', 'length', 'molecular_weight','viral'])

#print(df_unal.head())

# Unir el dataFrame con la columna antitumoral
df_merged = pd.merge(df_unal, df_excel_antitumoral, left_on='sequence', right_on='Secuencia', how='left')
df_merged['Antitumoral'] = df_merged['Secuencia'].notna().astype(int)
df_merged=df_merged.drop('Secuencia',axis=1)

# Unir el dataframe con la columna antivih
df_merged_unal = pd.merge(df_merged, df_excel_antivih, left_on='sequence', right_on='Secuencia', how='left')
df_merged_unal['Anti-VIH'] = df_merged_unal['Secuencia'].notna().astype(int)
df_merged_unal=df_merged_unal.drop('Secuencia',axis=1)


#Calculo la secuencia mas larga
max_length = df_merged_unal['sequence'].str.len().max()
print(max_length)

# Mostrar el DataFrame filtrado
print(df_merged_unal)




35
       antigramneg  antigrampos  bacteriano  cancer  fungico  microbiano  \
0                1            1           1       0        1           1   
1                1            1           1       0        1           1   
2                1            1           1       0        1           1   
3                1            1           1       0        1           1   
4                1            1           1       0        1           1   
...            ...          ...         ...     ...      ...         ...   
31718            0            0           0       0        0           0   
31719            0            0           0       0        0           0   
31720            0            0           0       0        0           0   
31721            0            0           0       0        0           0   
31722            0            0           0       0        0           0   

                                  sequence  Antitumoral  Anti-VIH  
0               

In [ ]:
#Prueba para un registro en especifico

registro = df_merged_unal[df_merged_unal['sequence'] == 'GFREVLKGAAKAFVKTVAGHIANI']

print(registro)

     antigramneg  antigrampos  bacteriano  cancer  fungico  microbiano  \
537            0            0           1       0        0           1   

                     sequence  Antitumoral  Anti-VIH  
537  GFREVLKGAAKAFVKTVAGHIANI            0         0  


#STARPEP

In [ ]:
df_starpep = pd.read_csv(file_path_starpep, delimiter=',')

#Elimino columnas innecesarias
df_starpep = df_starpep.drop(columns=['Relationship', 'Reference'])


valores_a_mantener = ['Antimicrobial', 'Anticancer', 'Anti-Gram positive', 'Anti-Gram negative','Anti-HIV', 'Antifungal', 'Antitumor', 'Antibacterial']

df_starpep = df_starpep[df_starpep['Metadata'].isin(valores_a_mantener)]


df_starpep = pd.pivot_table(df_starpep, values='Metadata', index='Peptide',
                          columns='Metadata', aggfunc=lambda x: 1, fill_value=0)

print(df_starpep)

In [ ]:
print(df_starpep)

Metadata       Anti-Gram negative  Anti-Gram positive  Anti-HIV  \
Peptide                                                           
starPep_00000                   1                   1         1   
starPep_00001                   1                   1         0   
starPep_00002                   1                   1         1   
starPep_00003                   1                   1         1   
starPep_00004                   1                   1         1   
...                           ...                 ...       ...   
starPep_45083                   1                   1         0   
starPep_45084                   1                   1         0   
starPep_45087                   1                   0         0   
starPep_45091                   0                   0         0   
starPep_45099                   1                   1         0   

Metadata       Antibacterial  Anticancer  Antifungal  Antimicrobial  Antitumor  
Peptide                                        

In [ ]:
#!pip install Bio
from Bio import SeqIO

# Suponiendo que has subido un archivo llamado 'sequences.fasta'
fasta_file = "/content/drive/My Drive/PI 2.0/Peptides.fasta"  # Cambia esto por el nombre de tu archivo

# Leer el archivo .fasta
records = SeqIO.parse(fasta_file, "fasta")

# Crear listas para los datos
ids = []
sequences = []

# Extraer la información
for record in records:
    ids.append(record.id)
    sequences.append(str(record.seq))

# Crear un DataFrame
df_fasta = pd.DataFrame({
    'ID': ids,
    'Secuencia': sequences
})




# Unir los DataFrames por las columnas de ID correspondientes
df_merged = pd.merge(df_starpep, df_fasta, left_on='Peptide', right_on='ID', how='inner')

orden = ['ID', 'Secuencia', 'Antimicrobial', 'Anticancer', 'Anti-Gram positive', 'Anti-Gram negative','Anti-HIV', 'Antifungal', 'Antitumor', 'Antibacterial' ]

df_merged_starpep = df_merged[orden]

df_merged_starpep = df_merged_starpep.drop('ID', axis=1)

print(df_merged_starpep)


                                   Secuencia  Antimicrobial  Anticancer  \
0                 GIGAVLKVLTTGLPALISWIKRKRQQ              1           1   
1                    GIGKFLHSAKKFGKAFVGEIMNS              1           1   
2                              ILPWKWPWWPWRR              1           1   
3                          KWCFRVCYRGICYRRCR              1           1   
4      LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES              1           1   
...                                      ...            ...         ...   
22849                   YYHFWHRGVTKRSLSPHRPR              0           0   
22850                YYHFWHRGVTKRSLSPHRPRHSR              0           0   
22851                   YYNPLPHDCGRDNNTDICSR              0           0   
22852                               YYQQKPVA              0           0   
22853                               YYYYRRRR              0           1   

       Anti-Gram positive  Anti-Gram negative  Anti-HIV  Antifungal  \
0                       1   

In [ ]:
#Unir los dataframe de starpep con la nacional

import pandas as pd

df_merged_starpep = pd.read_csv(file_merged_starpep)
df_merged_unal = pd.read_csv(file_merged_unal)

df_merged_starpep = df_merged_starpep.drop(columns=['Unnamed: 0'])
df_merged_unal = df_merged_unal.drop(columns=['Unnamed: 0'])

df_merged_unal = df_merged_unal.rename(columns={'antigramneg': 'Anti-Gram negative',
                        'antigrampos': 'Anti-Gram positive',
                        'bacteriano':'Antibacterial',
                        'cancer':'Anticancer',
                        'fungico':'Antifungal',
                        'microbiano':'Antimicrobial',
                        'sequence':'Secuencia',
                        'Antitumoral':'Antitumor',
                        'Anti-VIH':'Anti-HIV'})

columnas_ordenadas = ['Secuencia', 'Antimicrobial','Antibacterial', 'Antifungal',
                       'Anti-HIV','Antitumor', 'Anticancer', 'Anti-Gram positive',
                      'Anti-Gram negative']

df_merged_starpep=df_merged_starpep[columnas_ordenadas]
df_merged_unal=df_merged_unal[columnas_ordenadas]

df=pd.concat([df_merged_starpep,df_merged_unal], ignore_index=True)

print(df_merged_unal.head())
print(df_merged_starpep.head())


print('---------')

print(df.head())

              Secuencia  Antimicrobial  Antibacterial  Antifungal  Anti-HIV  \
0     AAFRGCWTKNYSPKPCL              1              1           1         0   
1            AAGMGFFGAR              1              1           1         0   
2  AALKGCWTKSIPPKPCFGKR              1              1           1         0   
3  AALKGCWTKSIPPKPCSGKR              1              1           1         0   
4  AALRGCWTKSIPPKPCPGKR              1              1           1         0   

   Antitumor  Anticancer  Anti-Gram positive  Anti-Gram negative  
0          0           0                   1                   1  
1          0           0                   1                   1  
2          0           0                   1                   1  
3          0           0                   1                   1  
4          0           0                   1                   1  
                               Secuencia  Antimicrobial  Antibacterial  \
0             GIGAVLKVLTTGLPALISWIKRKRQQ         

In [ ]:
#Filtrar por las secuencias menores a 50

df_completo=df
print(df_completo.shape)

df= df[df['Secuencia'].str.len() <= 50]

# Mostrar el DataFrame resultante
print(df.shape)

(43830, 9)
(39656, 9)


In [ ]:
#Subir los csv al drive

df_completo.to_csv('dfGrande.csv', index=False)
df.to_csv('dfLimpio.csv', index=False)